In [1]:
cd ../..

/home/nazif/thesis/mirscribe-vcf


/home/nazif/.cache/pypoetry/virtualenvs/mirscribe-vcf-8VJm4tPg-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
# db engine
engine = create_engine('sqlite:///data/db/mirscribe.db')

# pyensembl db engine
pyensembl = create_engine('sqlite:////home/nazif/.cache/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.gtf.db')


In [3]:
genes = pd.read_sql("gene", pyensembl, columns=["gene_id", "gene_name", "seqname", "start", "end", "gene_biotype"])

genes = genes.replace(r'^\s*$', np.nan, regex=True)
genes = genes.infer_objects(copy=False)

genes.rename(columns={"seqname": "chr"}, inplace=True)

genes.head()


,gene_id,gene_name,chr,start,end,gene_biotype
0,ENSG00000223972,DDX11L1,1,11869,14412,pseudogene
1,ENSG00000227232,WASH7P,1,14363,29806,pseudogene
2,ENSG00000243485,MIR1302-10,1,29554,31109,lincRNA
3,ENSG00000237613,FAM138A,1,34554,36081,lincRNA
4,ENSG00000268020,OR4G4P,1,52473,54936,pseudogene


In [4]:
expected_chroms = [str(i) for i in range(1, 23)] + ['X', 'Y']
len_before = len(genes)
len_expected_chroms = len(expected_chroms)
# drop patch chromosomes
genes = genes[genes.chr.isin(expected_chroms)]

print(f"dropped {len_before - len(genes)} genes from patches")

dropped 5941 genes from patches


# add biomart data

In [5]:
e75 = pd.read_csv("data/biomart/ensembl75_g37.tsv", sep="\t")
colnames = {'Gene stable ID': 'gene_id',
            'EntrezGene ID': 'entrez_id',
            'Gene description': 'gene_description',
            'Gene name': 'gene_name',
            'Gene type': 'biomart_biotype',
            }

e75 = e75.rename(columns={"Ensembl Gene ID": "gene_id"})


e112 = pd.read_csv("data/biomart/ensembl112_g37.tsv", sep="\t")
colnames = {'Gene stable ID': 'gene_id',
            'NCBI gene (formerly Entrezgene) ID': 'entrez_id',
            'Gene description': 'gene_description',
            'Gene type': 'biomart_biotype',
            }

e112 = e112.rename(columns=colnames)

In [6]:
pyen = set(genes.gene_id.tolist())
e75s = set(e75.gene_id.tolist())
e112s = set(e112.gene_id.tolist())

# compare 
print(f"there are {len(pyen - e75s)} genes in pyensembl75 not in e75")
print(f"there are {len(e75s - pyen)} genes in e75 not in pyensembl75")
print(f"there are {len(pyen & e75s)} genes in both")
print("######################")
print(f"there are {len(pyen - e112s)} genes in pyensembl75 not in e112")
print(f"there are {len(e112s - pyen)} genes in e112 not in pyensembl75")
print(f"there are {len(pyen & e112s)} genes in both")

there are 0 genes in pyensembl75 not in e75
there are 6366 genes in e75 not in pyensembl75
there are 57736 genes in both
######################
there are 0 genes in pyensembl75 not in e112
there are 5941 genes in e112 not in pyensembl75
there are 57736 genes in both


whole pipeline is done with pyensembl75, therefore i must use biomart export from ensembl 112. They're fully compatible

In [7]:
e112.entrez_id.value_counts()

entrez_id
80864.0        20
259215.0       17
100169763.0    16
554313.0       16
8367.0         16
               ..
149837.0        1
200634.0        1
6718.0          1
29125.0         1
100653067.0     1
Name: count, Length: 25788, dtype: int64

In [8]:
e112[e112.entrez_id == 80864.0]

,gene_id,entrez_id,gene_description,biomart_biotype
24600,ENSG00000206329,80864.0,palmitoyl-protein thioesterase 2 [Source:HGNC ...,protein_coding
25563,ENSG00000254875,80864.0,NaN,protein_coding
26024,ENSG00000244444,80864.0,"EGF-like-domain, multiple 8 [Source:HGNC Symbo...",protein_coding
30170,ENSG00000206256,80864.0,palmitoyl-protein thioesterase 2 [Source:HGNC ...,protein_coding
33847,ENSG00000242038,80864.0,"EGF-like-domain, multiple 8 [Source:HGNC Symbo...",protein_coding
36844,ENSG00000227600,80864.0,palmitoyl-protein thioesterase 2 [Source:HGNC ...,protein_coding
37573,ENSG00000239974,80864.0,"EGF-like-domain, multiple 8 [Source:HGNC Symbo...",protein_coding
39416,ENSG00000236649,80864.0,palmitoyl-protein thioesterase 2 [Source:HGNC ...,protein_coding
39637,ENSG00000240389,80864.0,"EGF-like-domain, multiple 8 [Source:HGNC Symbo...",protein_coding
53506,ENSG00000168452,80864.0,palmitoyl-protein thioesterase 2 [Source:HGNC ...,protein_coding


In [9]:
e112_to_merge = e112[["gene_id","gene_description"]].drop_duplicates()
df = pd.merge(genes, e112_to_merge, on="gene_id", how="left")

df.head()

,gene_id,gene_name,chr,start,end,gene_biotype,gene_description
0,ENSG00000223972,DDX11L1,1,11869,14412,pseudogene,DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 l...
1,ENSG00000227232,WASH7P,1,14363,29806,pseudogene,WAS protein family homolog 7 pseudogene [Sourc...
2,ENSG00000243485,MIR1302-10,1,29554,31109,lincRNA,microRNA 1302-10 [Source:HGNC Symbol;Acc:38233]
3,ENSG00000237613,FAM138A,1,34554,36081,lincRNA,"family with sequence similarity 138, member A ..."
4,ENSG00000268020,OR4G4P,1,52473,54936,pseudogene,"olfactory receptor, family 4, subfamily G, mem..."


In [10]:
df.isna().sum()

gene_id                 0
gene_name               0
chr                     0
start                   0
end                     0
gene_biotype            0
gene_description    21297
dtype: int64

In [11]:
df["gene_description"] = df["gene_description"].fillna("no_description")
# df["entrez_id"] = df["entrez_id"].fillna(-1).astype(int)


In [12]:
df.isna().sum()

gene_id             0
gene_name           0
chr                 0
start               0
end                 0
gene_biotype        0
gene_description    0
dtype: int64

# adding oncokb data

In [13]:
kb = pd.read_csv("data/oncokb/oncokb.csv")
kb = kb.rename(columns={"gene_symbol": "gene_name", "ensembl_gene_id": "gene_id", "is_oncogene": "is_oncogene", "is_tumor_suppressor": "is_tumor_suppressor", "is_cosmic_tier_1": "is_cosmic_tier_1"})

# drop columns where all bool values are false
cols = ["is_oncogene", "is_tumor_suppressor", "is_cosmic_tier_1"]
kb = kb[~(kb[cols] == False).all(axis=1)]

kb.head()



,gene_name,gene_id,is_oncogene,is_tumor_suppressor,is_cosmic_tier_1
0,ABL1,ENSG00000097007,True,False,True
1,AKT1,ENSG00000142208,True,False,True
2,ALK,ENSG00000171094,True,False,True
3,AMER1,ENSG00000184675,False,True,True
4,APC,ENSG00000134982,False,True,True


# checking if all gene ids are found in our db

In [14]:
kb.gene_id.isin(df.gene_id).value_counts()

gene_id
True     925
False     13
Name: count, dtype: int64

In [15]:
missing_gene_ids = kb[~kb.gene_id.isin(df.gene_id)].gene_name
missing_gene_ids

146       H3C2
238      IKBKE
394       CCNQ
453     H2AC17
697      GTF2I
700       H4C9
715        IGH
716        IGK
717        IGL
741      MLLT6
816       SSX4
820      TAF15
1005     NCOA4
Name: gene_name, dtype: object

In [16]:
missing_id_but_found_gene_names = genes[genes.gene_name.isin(missing_gene_ids)].gene_name
missing_id_but_found_gene_names

4487     IKBKE
22142    GTF2I
24354     SSX4
31690    NCOA4
48265    TAF15
48365    MLLT6
Name: gene_name, dtype: object

In [17]:
print(f"These genes are missing in the genes table: {list(set(missing_gene_ids) - set(missing_id_but_found_gene_names))}")

These genes are missing in the genes table: ['H3C2', 'IGL', 'CCNQ', 'H2AC17', 'H4C9', 'IGK', 'IGH']


H2AC17 is whole locus

H4C9 not found in grch37

H3C2 is HIST1H3D in grch37

IG's are whole loci with more than one ENSG ids

CCNQ is FAM58A in g37

note: these are found from g37 ensembl website

In [18]:
temp_df = kb.drop(columns=["gene_name"])
df = pd.merge(df, temp_df, on="gene_id", how="left")
df.head()

,gene_id,gene_name,chr,start,end,gene_biotype,gene_description,is_oncogene,is_tumor_suppressor,is_cosmic_tier_1
0,ENSG00000223972,DDX11L1,1,11869,14412,pseudogene,DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 l...,NaN,NaN,NaN
1,ENSG00000227232,WASH7P,1,14363,29806,pseudogene,WAS protein family homolog 7 pseudogene [Sourc...,NaN,NaN,NaN
2,ENSG00000243485,MIR1302-10,1,29554,31109,lincRNA,microRNA 1302-10 [Source:HGNC Symbol;Acc:38233],NaN,NaN,NaN
3,ENSG00000237613,FAM138A,1,34554,36081,lincRNA,"family with sequence similarity 138, member A ...",NaN,NaN,NaN
4,ENSG00000268020,OR4G4P,1,52473,54936,pseudogene,"olfactory receptor, family 4, subfamily G, mem...",NaN,NaN,NaN


# adding details of genes above

In [19]:
kb[kb.gene_name.isin(missing_id_but_found_gene_names)]


,gene_name,gene_id,is_oncogene,is_tumor_suppressor,is_cosmic_tier_1
238,IKBKE,ENSG00000263528,True,False,False
697,GTF2I,ENSG00000263001,True,False,False
741,MLLT6,ENSG00000275023,False,False,True
816,SSX4,ENSG00000268009,False,False,True
820,TAF15,ENSG00000270647,False,False,True
1005,NCOA4,ENSG00000266412,False,False,True


In [20]:
df[df.gene_name.isin(missing_id_but_found_gene_names)]

,gene_id,gene_name,chr,start,end,gene_biotype,gene_description,is_oncogene,is_tumor_suppressor,is_cosmic_tier_1
4487,ENSG00000143466,IKBKE,1,206643791,206670223,protein_coding,inhibitor of kappa light polypeptide gene enha...,NaN,NaN,NaN
22142,ENSG00000077809,GTF2I,7,74071994,74175026,protein_coding,general transcription factor IIi [Source:HGNC ...,NaN,NaN,NaN
24354,ENSG00000204645,SSX4,X,48242863,48252785,protein_coding,"synovial sarcoma, X breakpoint 4 [Source:HGNC ...",NaN,NaN,NaN
31690,ENSG00000138293,NCOA4,10,51565108,51590734,protein_coding,nuclear receptor coactivator 4 [Source:HGNC Sy...,NaN,NaN,NaN
48265,ENSG00000172660,TAF15,17,34136459,34191619,protein_coding,"TAF15 RNA polymerase II, TATA box binding prot...",NaN,NaN,NaN
48365,ENSG00000108292,MLLT6,17,36861795,36886056,protein_coding,myeloid/lymphoid or mixed-lineage leukemia (tr...,NaN,NaN,NaN


In [21]:
df.loc[df.gene_name == 'IKBKE', 'is_oncogene'] = True
df.loc[df.gene_name == 'GTF2I', 'is_oncogene'] = True
df.loc[df.gene_name == 'MLLT6', 'is_cosmic_tier_1'] = True
df.loc[df.gene_name == 'SSX4', 'is_cosmic_tier_1'] = True
df.loc[df.gene_name == 'TAF15', 'is_cosmic_tier_1'] = True
df.loc[df.gene_name == 'NCOA4', 'is_cosmic_tier_1'] = True

In [22]:
kb[kb.gene_name.isin(['CCNQ',"H3C2"])]

,gene_name,gene_id,is_oncogene,is_tumor_suppressor,is_cosmic_tier_1
146,H3C2,ENSG00000286522,False,False,True
394,CCNQ,ENSG00000262919,False,True,False


In [23]:
df.loc[df.gene_name == 'HIST1H3D', 'is_cosmic_tier_1'] = True
df.loc[df.gene_name == 'FAM58A', 'is_tumor_suppressor'] = True


# checking NA

In [24]:
df.isna().sum()

gene_id                    0
gene_name                  0
chr                        0
start                      0
end                        0
gene_biotype               0
gene_description           0
is_oncogene            56809
is_tumor_suppressor    56810
is_cosmic_tier_1       56806
dtype: int64

In [25]:
df.fillna(False, inplace=True)
df.head()

/tmp/ipykernel_29111/809247794.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(False, inplace=True)


,gene_id,gene_name,chr,start,end,gene_biotype,gene_description,is_oncogene,is_tumor_suppressor,is_cosmic_tier_1
0,ENSG00000223972,DDX11L1,1,11869,14412,pseudogene,DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 l...,False,False,False
1,ENSG00000227232,WASH7P,1,14363,29806,pseudogene,WAS protein family homolog 7 pseudogene [Sourc...,False,False,False
2,ENSG00000243485,MIR1302-10,1,29554,31109,lincRNA,microRNA 1302-10 [Source:HGNC Symbol;Acc:38233],False,False,False
3,ENSG00000237613,FAM138A,1,34554,36081,lincRNA,"family with sequence similarity 138, member A ...",False,False,False
4,ENSG00000268020,OR4G4P,1,52473,54936,pseudogene,"olfactory receptor, family 4, subfamily G, mem...",False,False,False


# editing is_cosmic_tier_1 column

is_cosmic_tier_1 column indicates if genes are listed in cosmic tier 1 genes.

COSMIC tier 1 genes include oncogenes, tumor suppressors and fusion genes.

Our data has oncogene and tsupp data but not fusion data.

Therefore I'll derive is_fusion column from genes that are in tier 1 but neither oncogene nor tsupp.

Then I'll drop tier 1 column because it will be irrelevant.

In [26]:
df["is_fusion"] = ~df.is_oncogene & ~df.is_tumor_suppressor & df.is_cosmic_tier_1
df.drop(columns=("is_cosmic_tier_1"), inplace=True)

# adding intogen data

In [28]:
into = pd.read_json("data/intogen/driver_genes.json")

into.head()

,gene_name,is_driver
0,ACVR1,True
1,ACVR2A,True
2,ARAF,True
3,ASXL2,True
4,BAP1,True


In [29]:
into.gene_name.isin(df.gene_name).value_counts()

gene_name
True     86
False     1
Name: count, dtype: int64

In [30]:
into[~into.gene_name.isin(df.gene_name)].gene_name

78    TENT5C
Name: gene_name, dtype: object

FAM46C is the grch37 name of TENT5C

source: https://www.ensembl.org/Homo_sapiens/Gene/Summary?g=ENSG00000183508;r=1:117606048-117628389;t=ENST00000369448

In [31]:
df = pd.merge(df, into, on="gene_name", how="left")

In [32]:
df.loc[df.gene_name == "FAM46C", "is_driver"] = True

In [33]:
df["is_driver"] = df["is_driver"].fillna(False)

/tmp/ipykernel_29111/2844565991.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["is_driver"] = df["is_driver"].fillna(False)


In [34]:
df.isna().sum()

gene_id                0
gene_name              0
chr                    0
start                  0
end                    0
gene_biotype           0
gene_description       0
is_oncogene            0
is_tumor_suppressor    0
is_fusion              0
is_driver              0
dtype: int64

# save to db

In [35]:
from sqlalchemy import create_engine

# db engine
engine = create_engine('sqlite:///data/db/mirscribe.db')

df.to_sql(name="genes", con=engine, if_exists="replace", index=False)

57736